In [119]:
import pandas as pd

In [157]:
adrc_before = pd.read_excel('../v-beforeUpload/adrc.xlsx', dtype=str)
adrc_after = pd.read_excel('../v-5-june/adrc.xlsx', dtype=str)

In [158]:
adrc_after.head()

,Address Number,from,Address Version,To,Title,Name,Name 2,Name 3,Name 4,Converted name (with form of address),...,Type of Delivry Service,Number of Delivery Service,County code,County,Township code,Township,County.1,Township.1,Business Purpose Completed Flag,Address notes
0,73364,0001-01-01,NaN,9999-12-31 00:00:00,0008,ALFA LAVAL INDIA PRIVATE LIMITED,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73379,0001-01-01,NaN,9999-12-31 00:00:00,0008,AQUA ENGINEERING SERVICES,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73385,0001-01-01,NaN,9999-12-31 00:00:00,0008,ANEST IWATA MOTHERSON,PVT.LTD.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73400,0001-01-01,NaN,9999-12-31 00:00:00,0008,AAF INDIA PVT LTD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73406,0001-01-01,NaN,9999-12-31 00:00:00,0008,BANGALORE DOR-TECH,PVT LTD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# adrc_after['key'] = adrc_after.apply(lambda row: str(row['Address Number']) + str(row['Person Number']) + str(row['Sequence Number']), axis=1)

In [ ]:
# adrc_before['key'] = adrc_before.apply(lambda row: str(row['Address Number']) + str(row['Person Number']) + str(row['Sequence Number']), axis=1)

In [ ]:
adrc_before.set_index('Address Number', inplace=True)
adrc_after.set_index('Address Number', inplace=True)

In [ ]:
adrc_before, adrc_after = adrc_before.align(adrc_after, join='inner', axis=0)

In [ ]:
diffs = adrc_before.compare(
    adrc_after,
    keep_equal=True,      # only show changed cells
    keep_shape=True,       # retain full shape if you want blanks where no change
    result_names=('Before','After')
)

In [143]:
diffs.columns = [f"{col[0]} ({col[1]})" for col in diffs.columns]

In [ ]:
diffs.to_excel('../adrc_allData_changes.xlsx')
# diffs.to_excel('../adrc_changes.xlsx')

In [ ]:
f = pd.read_excel('../adrc_allData_changes.xlsx', sheet_name='Sheet3')

In [148]:
f.head()

,Name,email
0,3M Electro & Communication,pbhutani@mmm.com
1,3M Electro & Communication,srjanak@mmm.com
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM
4,AAALAC INTERNATIONAL,msprouse@aaalac.org


In [153]:
def func(df):
    n = df['Name'].split()
    
    for i in n:
        if i.lower() in df['email'].lower():
            return True
    return False

In [154]:
f['new'] = f.apply(lambda row: func(row), axis=1)

In [155]:
f

,Name,email,new
0,3M Electro & Communication,pbhutani@mmm.com,False
1,3M Electro & Communication,srjanak@mmm.com,False
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM,True
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM,True
4,AAALAC INTERNATIONAL,msprouse@aaalac.org,True
...,...,...,...
2828,ZELLE BIOTECHNOLOGY PVT LTD,accounts@zellebiotech.com,True
2829,ZELLE BIOTECHNOLOGY PVT LTD,orders@zellebiotech.com,True
2830,ZELLE BIOTECHNOLOGY PVT LTD,service@zellebiotech.com,True
2831,ZoomInfo Technologies LLC,ar@zoominfo.com,True


In [156]:
f.to_csv('email.csv')